# Process Handbook

This notebook processes the example handbook (CAHRC_HR_Manual.txt).  This is done in a simple fashion using the following heuristic: If a line of text consisting of less than 5 words is followed by paragraphs of text the assume the line of text with less than 5 words is a topic (i.e. the topic of a question an employee might ask) and that the paragraphs of text are the answer to that question (called action_text for the lack of a better term).

When a topic and action_text are found these are stored in Cloud Datastore as a key-value pair with the topic as the key and the action_text as the value.

In [1]:
!pip uninstall -y google-cloud-datastore

Skipping google-cloud-datastore as it is not installed.


In [2]:
!pip install google-cloud-datastore==1.2.0

    100% |████████████████████████████████| 51kB 2.3MB/s ta 0:00:011
    100% |████████████████████████████████| 51kB 16.5MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 20.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for gapic-google-cloud-datastore-v1 ... done
  Stored in directory: /content/.cache/pip/wheels/79/a9/0f/1b6929a5f1961d22553efa844ea60c589bffcf6e1bd019f7c9
  Running setup.py bdist_wheel for ply ... done
  Stored in directory: /content/.cache/pip/wheels/f2/21/c0/f0056cc96847933daa961a19eb59a2ecd0228fdbe3376e7a68
  Running setup.py bdist_wheel for proto-google-cloud-datastore-v1 ... done
  Stored in directory: /content/.cache/pip/wheels/bd/ce/33/8b769968db3761c42c7a91d8a0dbbafc50acfa0750866c8abd
Successfully built gapic-google-cloud-datastore-v1 ply proto-google-cloud-datastore-v1
google-cloud-dataflow 2.0.0 has requirement httplib2<0.10,>=0.8, but you'll have httplib2 0.12.0 which is incompatible.
google-cloud-dataflow 2.0.0 has requirement proto-googl

  Found existing installation: google-cloud-core 0.28.1
    Uninstalling google-cloud-core-0.28.1:
      Successfully uninstalled google-cloud-core-0.28.1
  Found existing installation: proto-google-cloud-datastore-v1 0.90.0
    Uninstalling proto-google-cloud-datastore-v1-0.90.0:
      Successfully uninstalled proto-google-cloud-datastore-v1-0.90.0


Hit Reset Session > Restart, then resume with the following cells. 

In [1]:
from google.cloud import datastore

In [2]:
datastore_client = datastore.Client()

In [3]:
employee_handbook = open('CAHRC_HR_Manual.txt', 'r')
while True:
  
  topic = employee_handbook.readline()
  if not(topic):
    break
  
  if (topic != '\r\n') and (len(topic.split(' ')) < 5):
  
    action_text = ''
        
    last_line = ''
    line = employee_handbook.readline()
    
    while (last_line != '\r\n') and (line != '\r\n') and (len(line.split(' ')) > 5):
      
      action_text += line
      last_line = line
      line = employee_handbook.readline()
      
    if action_text != '':
      
      kind = 'Topic'
      topic_key = datastore_client.key(kind, topic.strip().lower())

      topic = datastore.Entity(key=topic_key)
      topic['action_text'] = action_text

      datastore_client.put(topic)

      print('Saved {}: {}'.format(topic.key.name, topic['action_text']))

Saved employment equity: [THE ORGANIZATION] is an equal opportunity employer and employs personnel without regard to race, ancestry, place of origin, colour, ethnic origin, language, citizenship, creed, religion, gender, sexual orientation, age, marital status, physical and/or mental handicap or financial ability. While remaining alert and sensitive to the issue of fair and equitable treatment for all, [THE ORGANIZATION] has a special concern with the participation and advancement of members of four designated groups that have traditionally been disadvantaged in employment: women, visible minorities, aboriginal peoples and persons with disabilities.

Saved recruitment and selection: All employment opportunities at [the organization]  are posted for a minimum 10 working day period.  They are posted on [THE ORGANIZATION]’s website and on the websites of affiliated organizations.  Occasionally, they are posted on employment websites or with an employment agency.  Applications are encourag

Saved resignation: After completion of the first ninety (90) days of the probationary period, employees must give the Employer two (2) weeks’ notice of resignation. The Employer may waive the resignation notice period in whole or in part at any time by providing payment of regular wages for the period so waived.

Saved layoff: Operation requirements are subject to change based on workload and the funding levels received on an annual basis.  All efforts will be made to keep staff in a position similar, in scope and salary, to that they have become accustom to.  If the organization is unable to do this, then employees will receive one (1) week notice for each year of service, as required by the Ontario Labour Code.  For employees who have a minimum of six years of service, this amount will be augmented by one (1) week of severance pay (or equivalent notice) for each year of service.

Saved employer property: Upon termination of employment for any reason, all items of any kind created or 